In [1]:
import Random

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [3]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()

    endstate,steps = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps)

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    println(steps)
    println(endstate)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [5]:
@time calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^7,10^10,Random.MersenneTwister())

Flux Balance on x
0.00031319164049951567
Flux Balance on y
0.00013953127062508962
Cov Balance on x,x
0.023931267283140663
Cov Balance on y,y
0.020958983231138834
Cov Balance on y,x
0.02309952596828269
99990704
[6, 26]
6.620477551872634
26.425457567646053
  6.988082 seconds (405 allocations: 32.266 KiB)


BalanceEqData(6.620477551872634, 26.425457567646053, 1.513071513306529e6, 8.807075553074017e6, 9.476664939115553e7, 1.3645067655454889e7, 6.610576515960011, 6.608506786878176, 26.434027147512705, 26.43771603562358, 8.664177638546022e6, -1.129186799372544e6, 1.4024801076180933e7, -7.966473625131835e6, 1.3296546019598754e7, 3.465671055418409e7, 9.41257877326485e7, 5.609920430472373e7)

In [6]:
function reach_balance_test(initialstate,params,numsteps,maxsteps,rng,flux_thres,cov_thres,tries)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    
    while (trynum < tries) && (!thresholdmet)
        new_max = trunc(Int,maxsteps*(tries-trynum)/tries)
        statevector,steps = direct_gillespie!(statevector,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,new_max)
        if (relative_error(calcstor.meanRpX,calcstor.meanRmX) < flux_thres) &&  (relative_error(calcstor.meanRpY,calcstor.meanRmY) < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum) < cov_thres
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum) < cov_thres
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum) <cov_thres 
            thresholdmet = all([cbxx_comp,cbyy_comp,cbyx_comp])               
        end
        trynum += 1
        totalsteps += steps
        print("Balance Test for chained simulation ")
        println(trynum)
        print("Current steps: ")
        println(totalsteps)
        print("Means: ")
        println([calcstor.meanx,calcstor.meany])
        print("Flux Balance on x: ")
        println(relative_error(calcstor.meanRpX,calcstor.meanRmX))
    
        print("Flux Balance on y: ")
        println(relative_error(calcstor.meanRpY,calcstor.meanRmY))
    
        print("Cov Balance on x,x: ")
        println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                                calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
        print("Cov Balance on y,y: ")
        println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                                calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
        print("Cov Balance on y,x: ")
        println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                                0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    end
    print("Tries taken: ")
    println(trynum)
    print("Total Steps: ")
    println(totalsteps)
    print("Threshold met: ")
    println(thresholdmet)
return calcstor
end

reach_balance_test (generic function with 1 method)

In [7]:
calcs = reach_balance_test([100,400],[10.,1.,4.,1.,-1.,50.],10^5,10^8,Random.MersenneTwister(),0.01,0.03,20)

Balance Test for chained simulation 1
Current steps: 999333
Means: [6.606126642639144, 26.419459019012553]
Flux Balance on x: 0.0030589595301617886
Flux Balance on y: 0.002162135461012946
Cov Balance on x,x: 0.029610793490713598
Cov Balance on y,y: 0.0318173793988818
Cov Balance on y,x: 0.13177768273251714
Balance Test for chained simulation 2
Current steps: 1999997
Means: [6.603407994573085, 26.46817001714554]
Flux Balance on x: 0.002551423020636138
Flux Balance on y: 0.004413370861403356
Cov Balance on x,x: 0.003718538384012045
Cov Balance on y,y: 0.008198014587713387
Cov Balance on y,x: 0.0734446074540474
Balance Test for chained simulation 3
Current steps: 2998191
Means: [6.601619856195192, 26.41063379476098]
Flux Balance on x: 0.003470648000955143
Flux Balance on y: 0.0024493052160981022
Cov Balance on x,x: 0.009636660068603186
Cov Balance on y,y: 0.000489925918843021
Cov Balance on y,x: 0.05738700224975093
Balance Test for chained simulation 4
Current steps: 4001831
Means: [6.607

BalanceEqData(6.620694400044947, 26.454460826619282, 121065.30575151293, 707348.5344028153, 7.694567811632848e6, 1.1046972417267116e6, 6.608102165160554, 6.60860397184567, 26.43441588738268, 26.46653488331824, 695975.388231228, -89939.08406572246, 1.1352737988588954e6, -637568.2945375307, 1.0766984704086597e6, 2.783901552924912e6, 7.643485809685048e6, 4.541095195435582e6)